In [ ]:
%pip install googledriver

In [ ]:
import pandas as pd

import googledriver

In [ ]:
dataset_name = 'fertilizantes_sinteticos'
raiz = '../databases'

### Bronze

In [ ]:
# id_arquivo = '1dwglTy1GCPjTEZW1z7-Hb5_DQIwGmdHzOVBMw0FEIsg' # ID adquirida na URL
id_arquivo = '1gIhcXrQ54qiBhM64tq1Lk8_2S_lIdiqKNBto907bcY4'
extensao = 'xlsx' # Formato de exportação da planilha: xlsx, ods ou csv
url_gdrive = f'https://docs.google.com/spreadsheets/d/{id_arquivo}/export?format={extensao}&t?gid=0#gid=0' # Endereço específico para exportação de dados planilhares
# TODO: Não funciona muito bem

saida = f'{raiz}/bronze_anda_{dataset_name}.{extensao}'

googledriver.download(url_gdrive, saida, None)

### Prata

NPK na forma de fertilizante sintético incorporado ao solo (t)

In [ ]:
# Criação de dataframe vazio para uso posterior
df_prata = pd.DataFrame()
saida = f"{raiz}/Dados_fertilizantes_minerais_ANDA_2000_atual.xlsx"

# Abas das tabelas na planilha bronze
tabelas = pd.ExcelFile(saida, engine='openpyxl').sheet_names

In [ ]:
df_prata = (
    pd.concat([
        pd.read_excel(
            saida,
            sheet_name=fertilizante,
            header=2,
            nrows=28,
            na_values=pd.NA
        )
        .query('uf != "Brasil"')
        .drop(columns='uf')
        # .dropna(axis='columns', thresh=10)
        .melt(
            id_vars='geocodigo',
            var_name='ano',
            value_name='valor'
        )
        .assign(**{
            'ano': lambda df: pd.to_datetime(df['ano'], format='%Y').dt.year, # Conversão de número inteiro para formato de data
            'fonte': 'Fertilizantes sintéticos',
            'elemento': fertilizante[len(fertilizante) - 1], # Criação de nova coluna de informação do que representa o dado
        })
        .rename(columns={'geocodigo': 'uf'})
        # .dropna(subset='valor')
        # .query('valor != 0') # Exclusão de registros que possuam valor zerado
        .set_index(['uf', 'ano', 'fonte', 'elemento'])
    for fertilizante in tabelas])
    .sort_index()
)

df_prata.to_parquet(f'{raiz}/prata_anda_{dataset_name}.parquet')

df_prata

# Ouro

## Elementos químicos na forma de fertilizante entregue ao mercado (t)

### Parâmetros

- Teor (%)

### Cálculos:

- Valor Estadual N * (Parâmetro N/100) * ((100 - fertilizantes_sinteticos)/100)

- Valor Estadual P2o5 * (Parâmetro P)/100

- Valor Estadual K2o * (Parâmetro K)/100


Brasil = mil t

In [ ]:
parametro_N = 100
parametro_P = 43.7
parametro_K = 83

# Perdas por volatilização do N
perdas_fertiliz_sintetico = 0

In [ ]:
gold_df = (
    pd.read_parquet(f'{raiz}/prata_anda_{dataset_name}.parquet')
        .reset_index()
        .assign(**{
            'valor': lambda df: df.apply(
                lambda linha: linha.valor*(parametro_N/100)*((100 - perdas_fertiliz_sintetico)/100) if linha.elemento == 'N'
                                     else linha.valor*(parametro_P/100) if linha.elemento == 'P'
                                     else linha.valor*(parametro_K/100),
                axis = 1
            )
        })
        .set_index(['uf', 'ano', 'fonte', 'elemento'])
        .squeeze()
    #     .replace({
    #         'Fert_Sint_N': 'N',
    #         'Fert_Sint_P': 'P',
    #         'Fert_Sint_K': 'K'
    #     })
    #   .convert_dtypes()
    )

gold_df.to_csv(f'{raiz}/ouro_npk_{dataset_name}.csv')

gold_df

In [ ]:
(gold_df.to_json(
    f'{raiz}/ouro_npk_{dataset_name}.json',
    orient='records',
    date_format='iso',
    date_unit='s',
    indent=4,
    force_ascii=False
            )
 )